<a href="https://colab.research.google.com/github/farhnsaptr/DIF62130_B_25_2311532018/blob/main/Praktikum3/BernoulliNB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: import pandas, sklearn bernoulliNB, standardscaler, accuracy_score, and train_test_split

import pandas as pd
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


In [2]:
dataset_dict = {
    'Outlook': ['sunny', 'sunny', 'overcast', 'rainy', 'rainy', 'rainy', 'overcast', 'sunny', 'sunny', 'rainy', 'sunny', 'overcast', 'overcast', 'rainy', 'sunny', 'overcast', 'rainy', 'sunny', 'sunny', 'rainy', 'overcast', 'rainy', 'sunny', 'overcast', 'sunny', 'overcast', 'rainy', 'overcast'],
    'Temperature': [85.0, 80.0, 83.0, 70.0, 68.0, 65.0, 64.0, 72.0, 69.0, 75.0, 75.0, 72.0, 81.0, 71.0, 81.0, 74.0, 76.0, 78.0, 82.0, 67.0, 85.0, 73.0, 88.0, 77.0, 79.0, 80.0, 66.0, 84.0],
    'Humidity': [85.0, 90.0, 78.0, 96.0, 80.0, 70.0, 65.0, 95.0, 70.0, 80.0, 70.0, 90.0, 75.0, 80.0, 88.0, 92.0, 85.0, 75.0, 92.0, 90.0, 85.0, 88.0, 65.0, 70.0, 60.0, 95.0, 70.0, 78.0],
    'Wind': [False, True, False, False, False, True, True, False, False, False, True, True, False, True, True, False, False, True, False, True, True, False, True, False, False, True, False, False],
    'Play': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes']
}

In [3]:
df = pd.DataFrame(dataset_dict)
df.head(5)

,Outlook,Temperature,Humidity,Wind,Play
0,sunny,85.0,85.0,False,No
1,sunny,80.0,90.0,True,No
2,overcast,83.0,78.0,False,Yes
3,rainy,70.0,96.0,False,Yes
4,rainy,68.0,80.0,False,Yes


In [4]:
df = pd.get_dummies(df, columns=['Outlook'], prefix='', prefix_sep='', dtype=int)

df['Wind'] = df['Wind'].astype(int)
df['Play'] = (df['Play'] == 'Yes').astype(int)

df.head(5)

,Temperature,Humidity,Wind,Play,overcast,rainy,sunny
0,85.0,85.0,0,0,0,0,1
1,80.0,90.0,1,0,0,0,1
2,83.0,78.0,0,1,1,0,0
3,70.0,96.0,0,1,0,1,0
4,68.0,80.0,0,1,0,1,0


In [5]:
df ['Temperature'] = pd.cut(df['Temperature'], bins=[0,80,100], labels=['warm', 'hot'])
df ['Humidity'] = pd.cut(df['Humidity'], bins=[0,75,100], labels=['dry', 'humid'])

one_hot_columns = pd.get_dummies(df[['Temperature', 'Humidity']], drop_first=True, dtype=int)
df = df.drop(['Temperature', 'Humidity'], axis=1)
df = pd.concat([ one_hot_columns, df], axis=1)

df.head()

,Temperature_hot,Humidity_humid,Wind,Play,overcast,rainy,sunny
0,1,1,0,0,0,0,1
1,0,1,1,0,0,0,1
2,1,1,0,1,1,0,0
3,0,1,0,1,0,1,0
4,0,1,0,1,0,1,0


- Bagi data menjadi data latih dan data uji

In [20]:
X = df.drop('Play', axis=1)
y = df['Play']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle = False)

In [21]:
y_train.head()

,Play
0,0
1,0
2,1
3,1
4,1


# Main Mechanism
- Hitung probabilitias setiap kelas dalam data pelatihan
- Untuk setiap fitur dan kelas, hitung probabilitas fitur tersebut bernilai 1 dan 0 berdasarkan kelasnya.
- Untuk data baru: untuk setiap kelas, kalikan probabilitas dengan probabilitas setiap nilai fitur (0 atau 1) untuk kelas tersebut.
- Prediksi kelas dengan probabilitas hasil tertinggi.

# Training Steps
proses pelatihan untuk Bernoulli Naive Bayes melibatkan perhitungan probabilitas dari data pelatihan:
1. Class Probability Calculation: Untuk setiap kelas, hitung probabilitasnya: (Jumlah kejadian di kelas ini) / (Jumlah total kejadian)

In [22]:
# Count the occurrences of 1 and 0 in the 'Play' column
one_count = y_train.value_counts()[1]
print(f"Count of 1s in 'Play' column: {one_count}")
zero_count = y_train.value_counts()[0]
print(f"Count of 0s in 'Play' column: {zero_count}")
total_count = one_count + zero_count
print(f"Total count: {total_count}")

# Calculate the ratio
ratio_true = one_count / total_count
ratio_false = zero_count / total_count

print(f"Ratio of 1s in 'Play' column: {ratio_true}")
print(f"Ratio of 0s in 'Play' column: {ratio_false}")


Count of 1s in 'Play' column: 11
Count of 0s in 'Play' column: 8
Total count: 19
Ratio of 1s in 'Play' column: 0.5789473684210527
Ratio of 0s in 'Play' column: 0.42105263157894735


2. Feature Probability Calculation: Untuk setiap fitur dan setiap kelas, hitunglah:
- (Jumlah kejadian dimana fitur adalah 0 di kelas ini) / (Jumlah kejadian di kelas ini)
- (Jumlah kejadian dimana fitur adalah 1 di kelas ini) / (Jumlah kejadian di kelas ini)


In [46]:
def hitung_rasio(kejadian, total):
  return kejadian / total

temp_one = hitung_rasio(X_train['Temperature_hot'].value_counts()[1], X_train.value_counts().sum())
print(f"Ratio of 1s in 'Temperature_hot' column: {temp_one}")
temp_zero = hitung_rasio(X_train['Temperature_hot'].value_counts()[0], X_train.value_counts().sum())
print(f"Ratio of 0s in 'Temperature_hot' column: {temp_zero}")
print(" ")

humid_one = hitung_rasio(X_train['Humidity_humid'].value_counts()[1], X_train.value_counts().sum())
print(f"Ratio of 1s in 'Humidity_hot' column: {humid_one}")
humid_zero = hitung_rasio(X_train['Humidity_humid'].value_counts()[0], X_train.value_counts().sum())
print(f"Ratio of 0s in 'Humidity_hot' column: {humid_zero}")
print(" ")

wind_one = hitung_rasio(X_train['Wind'].value_counts()[1], X_train.value_counts().sum())
print(f"Ratio of 1s in 'Wind' column: {wind_one}")
wind_zero = hitung_rasio(X_train['Wind'].value_counts()[0], X_train.value_counts().sum())
print(f"Ratio of 0s in 'Wind' column: {wind_zero}")
print(" ")

overcast_one = hitung_rasio(X_train['overcast'].value_counts()[1], X_train.value_counts().sum())
print(f"Ratio of 1s in 'overcast' column: {overcast_one}")
overcast_zero = hitung_rasio(X_train['overcast'].value_counts()[0], X_train.value_counts().sum())
print(f"Ratio of 0s in 'overcast' column: {overcast_zero}")
print(" ")

rainy_one = hitung_rasio(X_train['rainy'].value_counts()[1], X_train.value_counts().sum())
print(f"Ratio of 1s in 'rainy' column: {rainy_one}")
rainy_zero = hitung_rasio(X_train['rainy'].value_counts()[0], X_train.value_counts().sum())
print(f"Ratio of 0s in 'rainy' column: {rainy_zero}")
print(" ")

sunny_one = hitung_rasio(X_train['sunny'].value_counts()[1], X_train.value_counts().sum())
print(f"Ratio of 1s in 'sunny' column: {sunny_one}")
sunny_zero = hitung_rasio(X_train['sunny'].value_counts()[0], X_train.value_counts().sum())
print(f"Ratio of 0s in 'sunny' column: {sunny_zero}")

Ratio of 1s in 'Temperature_hot' column: 0.2631578947368421
Ratio of 0s in 'Temperature_hot' column: 0.7368421052631579
 
Ratio of 1s in 'Humidity_hot' column: 0.6842105263157895
Ratio of 0s in 'Humidity_hot' column: 0.3157894736842105
 
Ratio of 1s in 'Wind' column: 0.42105263157894735
Ratio of 0s in 'Wind' column: 0.5789473684210527
 
Ratio of 1s in 'overcast' column: 0.2631578947368421
Ratio of 0s in 'overcast' column: 0.7368421052631579
 
Ratio of 1s in 'rainy' column: 0.3157894736842105
Ratio of 0s in 'rainy' column: 0.6842105263157895
 
Ratio of 1s in 'sunny' column: 0.42105263157894735
Ratio of 0s in 'sunny' column: 0.5789473684210527


Sekarang, kita coba menggunakan sklearn.naive_bayes BernoulliNB

In [47]:
nb_clf = BernoulliNB()
nb_clf.fit(X_train, y_train)

BernoulliNB()

Prediksi data uji dan cek akurasi

In [48]:
y_pred = nb_clf.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.5555555555555556
